In [1]:
import os
Root = r"C:\Users\Chanakya\OneDrive\Desktop\SAP Training\Project\Dataset"
os.chdir(Root)

In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [15]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [16]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [17]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"C:\Users\Chanakya\OneDrive\Desktop\SAP Training\Project\Dataset/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [18]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [19]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342965e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174108e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216802e-05,  3.46632514e-05,  1.62844426e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575644e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750732e+01, ...,
         1.61086471e-04,  1.04962463e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151546e-04,  5.55269769e-04,  4.47782222e-04]])

In [20]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [21]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [22]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [23]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [24]:
#Predict for the test set
y_pred=model.predict(x_test)

In [25]:
y_pred

array(['happy', 'fearful', 'disgust', 'happy', 'disgust', 'fearful',
       'disgust', 'disgust', 'disgust', 'fearful', 'happy', 'fearful',
       'fearful', 'happy', 'disgust', 'happy', 'calm', 'disgust',
       'disgust', 'calm', 'disgust', 'disgust', 'disgust', 'calm',
       'happy', 'happy', 'disgust', 'happy', 'fearful', 'fearful',
       'happy', 'disgust', 'happy', 'fearful', 'happy', 'disgust',
       'disgust', 'fearful', 'calm', 'disgust', 'happy', 'disgust',
       'disgust', 'calm', 'fearful', 'disgust', 'disgust', 'disgust',
       'disgust', 'happy', 'fearful', 'fearful', 'fearful', 'fearful',
       'happy', 'fearful', 'disgust', 'happy', 'disgust', 'calm',
       'disgust', 'calm', 'disgust', 'happy', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'happy', 'fearful', 'fearful',
       'fearful', 'fearful', 'fearful', 'disgust', 'fearful', 'happy',
       'disgust', 'fearful', 'disgust', 'fearful', 'fearful', 'calm',
       'disgust', 'disgust', 'disgust', 

In [28]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 55.21%


In [29]:
from sklearn.metrics import accuracy_score, f1_score

In [30]:
f1_score(y_test, y_pred,average=None)

array([0.44155844, 0.546875  , 0.61052632, 0.5952381 ])

In [31]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,fearful
2,happy,disgust
3,happy,happy
4,disgust,disgust
5,calm,fearful
6,happy,disgust
7,happy,disgust
8,disgust,disgust
9,happy,fearful


In [32]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [34]:
#Input to predict the emotion(audio file)
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(r"C:\Users\Chanakya\OneDrive\Desktop\SAP Training\Project\Dataset/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')

In [37]:
print(prediction)

['disgust']
